# This is a demo to get you more familiar with how to merge tabular data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np

from brickstudy import foundation

### now we will make some sample just data just as an example

In [ ]:

# initialize list of lists
data1 = [['frank', 11], ['sushi', 15], ['julie', 14]]
 
# Create the pandas DataFrame
df1 = pd.DataFrame(data1, columns=['Name', 'Age'])
df1

In [ ]:
data2 = [['frank', 'Zaandam'], ['julie', 'Rotterdam'], ['sushi', 'Amstersdam']]
 
# Create the pandas DataFrame
df2 = pd.DataFrame(data2, columns=['Name', 'City'])
df2

# We just made two datasets. Now we will merge them. Just sticking them together randomly will not work. We want to use the names as a key. We mant pandas to reorganize the dataframes, based on that key, so all the right info goes with the right person. 

In [ ]:
merged = df1.merge(df2, left_on='Name', right_on='Name')

In [ ]:
merged

## Ok that was easy. Let's move on to what you have...excel spreadsheets with keys named at random, which need to be matched (your computer is not psychic)

In [ ]:
# Scenario A: different common key

# initialize list of lists
data3 = [['frank', 11], ['sushi', 15], ['julie', 14]]
 
# Create the pandas DataFrame
df3 = pd.DataFrame(data1, columns=['Not_the_same', 'Age'])
df3 

In [ ]:
data4 = [['frank', 'Zaandam'], ['julie', 'Rotterdam'], ['sushi', 'Amstersdam']]
 
# Create the pandas DataFrame
df4 = pd.DataFrame(data2, columns=['Key_is_notCommon', 'City'])
df4

In [ ]:
merged2 = df3.merge(df4, left_on='Not_the_same', right_on='Key_is_notCommon')

In [ ]:
merged2

# So that worked but now we have an extra column we don't need. We can drop it, but better to just name all the keys the same, and never anything else the same as the key

# Now about excel spreadsheets. If we have a key for all the sheets, we can write a function to simply read each sheet as a csv, then merge them


But we need to make sure they key columns actually match!
Let's see an example when they do not by adding a row to df1

In [ ]:
df1

In [ ]:
df1.loc[len(df1.index)] = ['amy', 89]
df1

df2

In [ ]:

merged5 = df1.merge(df2, left_on='Name', right_on='Name')
merged5

# Where did poor Amy go? She was lost because we left the merge pattern on default.
Let's try again

In [ ]:
merged5 = df1.merge(df2, how= 'outer',left_on='Name', right_on='Name')
merged5

# This is just the basic level you need to know to start attaching more data once I am gone. I will build some functions to automate you tabular dta attachment this weeend.

# part #2: dealing with Excel

In [ ]:
# Read Excel file with multiple sheets
xls = pd.ExcelFile("sample_synthetic_data/example_excel.xlsx")
# Get the list of sheet names
sheet_names = xls.sheet_names
# Print the sheet names
print(sheet_names)


In [ ]:
excel_file = pd.read_excel("sample_synthetic_data/example_excel.xlsx", sheet_name= sheet_names )

In [ ]:
print(type(excel_file))


So now we made a dictionary of the excel file. Let's access all parts of it

In [ ]:
len(sheet_names)

In [ ]:
sheet1 = excel_file[sheet_names[0]]
sheet1

In [ ]:
sheet2 = excel_file[sheet_names[1]]
sheet3 = excel_file[sheet_names[1]]


In [ ]:
sumsheet = sheet1.merge(sheet2, on='row_key')
sumsheet

In [ ]:
sheet3

In [ ]:
sumsheet = sumsheet.merge(sheet3, on='row_key')

In [ ]:
sumsheet

# SO what was the bad idea here...we merged blank columsn with nothing in them...could we have dropped our empty columns beforehand?

In [ ]:
sumsheet.dropna(how='all', axis=1, inplace=True)

In [ ]:
sumsheet

In [ ]:
# So now, let's put this all togehter in a function:

def csv_my_excel(excel_book_name, keyname):
    # Read Excel file with multiple sheets
    xls = pd.ExcelFile(excel_book_name)
    # Get the list of sheet names
    sheet_names = xls.sheet_names
    sheet_number= len(sheet_names)
    excel_file = pd.read_excel(excel_book_name, sheet_name= sheet_names)
    sheet_list = []
    for sheet_name in sheet_names:
        sheet = excel_file[sheet_name]
        sheet_list.append(sheet)
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=keyname,
                                            how='outer'), sheet_list)
    df_merged.dropna(how='all', axis=1, inplace=True)
    return df_merged
    

In [ ]:
csv_my_excel("sample_synthetic_data/example_excel.xlsx", "row_key")

let's call it from our  budding lirbary

In [ ]:
foundation.csv_my_excel("sample_synthetic_data/example_excel.xlsx", "row_key")